In [128]:
import pandas as pd
import numpy as np
import os

results_dir = "Analyzer Results"

input_dir = "TT Input"

output_dir = "Top Tester Results"

os.makedirs(output_dir, exist_ok=True)

In [129]:
# ...existing code...

dfs = {}
dfs_fit = {}

files = [file for file in os.listdir(results_dir) if file.endswith(('.xlsx', '.xls'))]

for file in files:
    file_path = os.path.join(results_dir, file)
    key = file.split('.')[0]
    if key.endswith('_FIT'):
        dfs_fit[key] = pd.read_excel(file_path)
    else:
        dfs[key] = pd.read_excel(file_path)

print("Raw dataframes:", list(dfs.keys()))
print("Fit dataframes:", list(dfs_fit.keys()))
print(f"Found {len(dfs)} raw and {len(dfs_fit)} fit dataframes in {results_dir}.")
if len(dfs) != len(dfs_fit):
    print("Warning: Mismatch between raw and fit dataframes!")

experiments = dfs[list(dfs.keys())[1]]['experiment'].unique().tolist()
variables = dfs[list(dfs.keys())[1]].columns[4:].tolist()
parameters = dfs[list(dfs.keys())[1]].columns[:4].tolist()
print(f"\nFound {len(experiments)} experiments, {len(variables)} variables and {len(parameters)} parameters:")
print(" "+', '.join(experiments))
print(" "+', '.join(variables))
print(" "+', '.join(parameters))

Raw dataframes: ['RESULTS', 'RESULTS_MERGED', 'RESULTS_MERGED_DATE', 'RESULTS_MERGED_EXP', 'RESULTS_MTT', 'RESULTS_MTT_MERGED', 'RESULTS_MTT_MERGED_DATE', 'RESULTS_MTT_MERGED_EXP', 'RESULTS_TT', 'RESULTS_TT_MERGED', 'RESULTS_TT_MERGED_DATE', 'RESULTS_TT_MERGED_EXP', 'SegmentOrder']
Fit dataframes: ['RESULTS_FIT', 'RESULTS_MERGED_DATE_FIT', 'RESULTS_MERGED_EXP_FIT', 'RESULTS_MERGED_FIT', 'RESULTS_MTT_FIT', 'RESULTS_MTT_MERGED_DATE_FIT', 'RESULTS_MTT_MERGED_EXP_FIT', 'RESULTS_MTT_MERGED_FIT', 'RESULTS_TT_FIT', 'RESULTS_TT_MERGED_DATE_FIT', 'RESULTS_TT_MERGED_EXP_FIT', 'RESULTS_TT_MERGED_FIT', 'SegmentOrder_FIT']
Found 13 raw and 13 fit dataframes in Analyzer Results.

Found 18 experiments, 9 variables and 4 parameters:
 ASR_control, gap_depth, tone_in_noise, gap_duration_4, gap_duration_8, gap_duration_10, gap_duration_20, gap_duration_50, offset_PPI_4, offset_PPI_6, offset_PPI_8, offset_PPI_10, offset_PPI_12, offset_PPI_14, offset_PPI_16, offset_PPI_18, offset_PPI_20, offset_PPI_50
 rea

In [130]:
# ...existing code...
before = len(dfs)
files = [file for file in os.listdir(input_dir) if file.endswith(('.xlsx', '.xls'))]

for file in files:
    file_path = os.path.join(input_dir, file)
    key = file.split('.')[0]
    dfs[key] = pd.read_excel(file_path)
    print(f"Loaded {file} into dataframe with key '{key}'.")

print("All dataframes:", list(dfs.keys()))

experiments = dfs[list(dfs.keys())[1]]['experiment'].unique().tolist()
variables = dfs[list(dfs.keys())[1]].columns[4:].tolist()
parameters = dfs[list(dfs.keys())[1]].columns[:4].tolist()
print(f"\nFound {len(experiments)} experiments, {len(variables)} variables and {len(parameters)} parameters:")
print(" "+', '.join(experiments))
print(" "+', '.join(variables))
print(" "+', '.join(parameters))

Loaded RESULTS_MT10.xlsx into dataframe with key 'RESULTS_MT10'.
Loaded RESULTS_MT20.xlsx into dataframe with key 'RESULTS_MT20'.
Loaded RESULTS_MT30.xlsx into dataframe with key 'RESULTS_MT30'.
Loaded RESULTS_T10.xlsx into dataframe with key 'RESULTS_T10'.
Loaded RESULTS_T20.xlsx into dataframe with key 'RESULTS_T20'.
Loaded RESULTS_T30.xlsx into dataframe with key 'RESULTS_T30'.
All dataframes: ['RESULTS', 'RESULTS_MERGED', 'RESULTS_MERGED_DATE', 'RESULTS_MERGED_EXP', 'RESULTS_MTT', 'RESULTS_MTT_MERGED', 'RESULTS_MTT_MERGED_DATE', 'RESULTS_MTT_MERGED_EXP', 'RESULTS_TT', 'RESULTS_TT_MERGED', 'RESULTS_TT_MERGED_DATE', 'RESULTS_TT_MERGED_EXP', 'SegmentOrder', 'RESULTS_MT10', 'RESULTS_MT20', 'RESULTS_MT30', 'RESULTS_T10', 'RESULTS_T20', 'RESULTS_T30']

Found 18 experiments, 9 variables and 4 parameters:
 ASR_control, gap_depth, tone_in_noise, gap_duration_4, gap_duration_8, gap_duration_10, gap_duration_20, gap_duration_50, offset_PPI_4, offset_PPI_6, offset_PPI_8, offset_PPI_10, offset_

In [131]:
# ...existing code...

from scipy.stats import mannwhitneyu

def compare_dataframes(df1, df2, parameters, variables):
    results = []
    # Get all unique parameter combinations in df1
    param_combos = df1[parameters].drop_duplicates()
    for _, combo in param_combos.iterrows():
        # Filter both dataframes for this parameter combo
        mask1 = (df1[parameters] == combo.values).all(axis=1)
        mask2 = (df2[parameters] == combo.values).all(axis=1)
        if mask1.sum() == 0 or mask2.sum() == 0:
            continue  # Skip if combo not present in both
        for var in variables:
            data1 = df1.loc[mask1, var].dropna()
            data2 = df2.loc[mask2, var].dropna()
            if len(data1) < 2 or len(data2) < 2:
                continue  # Need at least 2 samples for Mann-Whitney
            stat, p = mannwhitneyu(data1, data2, alternative='two-sided')
            results.append({
                **dict(zip(parameters, combo.values)),
                'variable': var,
                'statistic': stat,
                'p_value': p,
                'n_df1': len(data1),
                'n_df2': len(data2)
            })
    return pd.DataFrame(results)

In [132]:
# ...existing code...

import ast

def flatten_column(col):
    # Flattens a column of lists, stringified lists, or scalars into a single list of numbers
    out = []
    for item in col.dropna():
        if isinstance(item, list):
            out.extend(item)
        elif isinstance(item, str):
            try:
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    out.extend(parsed)
                elif isinstance(parsed, (int, float, np.integer, np.floating)):
                    out.append(parsed)
            except Exception:
                continue  # Ignore strings that can't be parsed
        elif isinstance(item, (int, float, np.integer, np.floating)):
            out.append(item)
        # Ignore non-numeric, non-list
    return pd.Series(out)

# ...existing code...

def compare_general_trends(df1, df2, variables):
    summary = []
    for var in variables:
        flat1 = flatten_column(df1[var])
        flat2 = flatten_column(df2[var])
        if len(flat1) > 0 and len(flat2) > 0:
            stats1 = {
                'mean': flat1.mean(),
                'median': flat1.median(),
                'std': flat1.std(),
                'n': flat1.count()
            }
            stats2 = {
                'mean': flat2.mean(),
                'median': flat2.median(),
                'std': flat2.std(),
                'n': flat2.count()
            }
            summary.append({
                'variable': var,
                'df1_mean': stats1['mean'],
                'df1_median': stats1['median'],
                'df1_std': stats1['std'],
                'df1_n': stats1['n'],
                'df2_mean': stats2['mean'],
                'df2_median': stats2['median'],
                'df2_std': stats2['std'],
                'df2_n': stats2['n'],
            })
        else:
            summary.append({
                'variable': var,
                'df1_mean': 'no data',
                'df1_median': 'no data',
                'df1_std': 'no data',
                'df1_n': len(flat1),
                'df2_mean': 'no data',
                'df2_median': 'no data',
                'df2_std': 'no data',
                'df2_n': len(flat2),
            })
    return pd.DataFrame(summary)

# ...existing code...

In [133]:
from scipy.stats import mannwhitneyu

def test_general_trend_significance(df1, df2, variables):
    results = []
    for var in variables:
        flat1 = flatten_column(df1[var])
        flat2 = flatten_column(df2[var])
        if len(flat1) > 1 and len(flat2) > 1:
            stat, p = mannwhitneyu(flat1, flat2, alternative='two-sided')
            results.append({
                'variable': var,
                'n_df1': len(flat1),
                'n_df2': len(flat2),
                'statistic': stat,
                'p_value': p
            })
        else:
            results.append({
                'variable': var,
                'n_df1': len(flat1),
                'n_df2': len(flat2),
                'statistic': 'no data',
                'p_value': 'no data'
            })
    return pd.DataFrame(results)

In [138]:
df1 = dfs['RESULTS_T30']
df2 = dfs['RESULTS_MT30']

In [139]:
result_df = compare_dataframes(df1, df2, parameters, variables)
print(result_df)

Empty DataFrame
Columns: []
Index: []


In [140]:
trend_df = compare_general_trends(df1, df2, variables)
print(trend_df)

       variable      df1_mean  df1_median      df1_std  df1_n      df2_mean  \
0  reactionTime     10.494071      10.000     1.896574    506     10.599532   
1      peakTime     31.023715      30.000     6.110811    506     30.384075   
2    difference     20.529644      20.000     6.403287    506     19.784543   
3     peakValue    134.675889     125.000    63.365713    506    121.775176   
4          PTPA    134.327787     124.920    62.508161    506    121.614926   
5          PTTA    122.229723     113.855    59.001448    506    109.582826   
6           RMS     52.534901      47.755    25.533483    506     47.617073   
7           tau    133.718300     134.850    48.113892    506    139.188087   
8           AUC  12893.197213   11091.885  6415.422542    506  12114.341499   

   df2_median      df2_std  df2_n  
0       10.00     2.019352   1281  
1       30.00     4.637739   1281  
2       20.00     5.076026   1281  
3      112.00    60.505857   1281  
4      112.68    59.832819   

In [141]:
sig_df = test_general_trend_significance(df1, df2, variables)
print(sig_df)
# Optionally save:

       variable  n_df1  n_df2  statistic   p_value
0  reactionTime    506   1281   309381.5  0.088257
1      peakTime    506   1281   338736.0  0.054507
2    difference    506   1281   348143.0  0.007205
3     peakValue    506   1281   363830.0  0.000053
4          PTPA    506   1281   363743.5  0.000055
5          PTTA    506   1281   365514.0  0.000025
6           RMS    506   1281   361403.5  0.000147
7           tau    506   1281   299643.0  0.012851
8           AUC    506   1281   350703.0  0.006776
